In [12]:
%pylab inline
from __future__ import division
import scipy as sp
from scipy import stats
from scipy.signal import argrelextrema
from scipy.interpolate import interp1d
import operator as op
from decimal import *
# import pickle
from mpl_toolkits.mplot3d import Axes3D
import mpld3
# mpld3.enable_notebook()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
%run ../cow.py
%run ../optimize_pkg_2.py

In [3]:
filepath_down = "../data_tables/hs_rs_downlink/n"
filepath_up = "../data_tables/hs_rs_uplink/n"

In [115]:
def xor_analysis_new(N, tSNR, rSNRdu, rSNR3, p_a1=10**(-9), p_a2=10**(-9), p_a3=10**(-9)):
    if rSNRdu > tSNR: return 0
    h_du = 10**((rSNRdu - tSNR)/10) # linear fade
    h_xor = 10**((rSNR3 - tSNR)/10)
    # Probability fade is bad
    p_f1 = p_f2 = 1 - np.exp(-h_du)
    p_f3 = 1 - np.exp(h_du-h_xor) if h_xor > h_du else 0

    p_link_1 = p_f1 + (1 - p_f1) * p_a1
    p_link_2 = p_f2 + (1 - p_f2) * p_a2
    
    test = []
    
    result = 0
    rv_gc = binom(N, 1 - p_f1)
    for gc in range(0, N+1):
        rv_ad = binom(gc, 1 - p_a1)
        for ad in range(0, gc+1):
            rv_ad_tilde = binom(ad, 1 - p_a2)
            bu = gc - ad
            rv_bu = binom(ad, 1-p_link_2)
            kad = np.arange(1, ad+1)
            s_bu_tilde = (1 - p_f3) + p_f3 * np.dot(rv_bu.pmf(kad), 1-np.power(p_f3, kad)) if p_f3 else 1
            # s_bu_tilde = (1 - p_f3) + p_f3 * sum([rv_bu.pmf(k) * (1 - p_f3**k) for k in range(1, ad+1)]) if p_f3 else 1
            q_bu_tilde = (1 - s_bu_tilde) + s_bu_tilde * p_a3
            for ad_tilde in range(0, ad+1):
                rv_ad_tilde_s = binom(ad_tilde, 1 - p_f3)
                rv_ad_hat_s = binom(ad - ad_tilde, 1 - p_f3)
                # if p_f3 = 0 then ad_tilde_s should = ad_tilde because ad_tilde_i should be empty
                for ad_tilde_s in range(0 if p_f3 else ad_tilde, ad_tilde+1):
                    # ad_tilde already succeeded
                    # if p_f3 = 0 then ad_hat_s should = ad - ad_tilde because ad_hat_i should be empty
                    for ad_hat_s in range(0 if p_f3 else ad-ad_tilde, ad-ad_tilde+1):
                        ad_s = ad_tilde_s + ad_hat_s
                        ad_i = ad - ad_tilde_s - ad_hat_s # ad - ad_s
                        rv_ads = binom(ad_s, 1 - p_link_2)
                        rv_adi = binom(ad_i, 1 - p_link_2)
                        q_ad_hat_s = p_a3
                        q_ad_hat_i = p_link_2**ad_s + (1 - p_link_2)**ad_s * p_a3
                        # This is the problem zone that doesn't support vectorization (?)
                        ks, ki = np.arange(1, ad_s+1), np.arange(0, ad_i+1)
                        f_e = p_link_2**ad_s + (np.dot(rv_ads.pmf(ks), np.power(p_f3, ks)) * np.dot(rv_adi.pmf(ki), np.power(p_f3, ki)) if p_f3 else 0)
                        q_e = f_e + (1 - f_e) * (1 - (1 - p_a3)**2)
                        rv_bu_tilde = binom(bu, 1 - p_a2)
                        bu_tilde = np.arange(0, bu+1)
                        log_pstates = np.log10(rv_gc.pmf(gc)) + np.log10(rv_ad.pmf(ad)) + np.log10(rv_ad_tilde.pmf(ad_tilde)) + np.log10(rv_ad_tilde_s.pmf(ad_tilde_s)) + np.log10(rv_ad_hat_s.pmf(ad_hat_s))
                        bu_tilde_res = np.dot(rv_bu_tilde.pmf(bu_tilde), np.multiply(np.power(1 - q_e, N-ad-bu_tilde), np.power(1 - q_bu_tilde, bu_tilde)))
                        res = 10**log_pstates * (1 - q_ad_hat_s)**ad_hat_s * (1 - q_ad_hat_i)**(ad - ad_tilde - ad_hat_s) * bu_tilde_res
                        result += res
                        # print res, result
                        # print gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, bu_tilde
                        # print "\n"
                        test.append([gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, res, rv_gc.pmf(gc)])
    return result, test

In [73]:
%%time
downbit = upbit = xorbit = 4*10**4/3
N = 7

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 18.2
rSNRdu = 1.4
rSNR3 = 1.5
xor_opt, histdata = xor_analysis_new(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

CPU times: user 2.92 s, sys: 51.6 ms, total: 2.97 s
Wall time: 3.31 s


/Users/leahdickstein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in log10


In [75]:
print 1 - xor_opt
histdata = matrix(histdata)
print np.where(histdata[:,-1] > 0.5)
print histdata[791]

8.27353852095e-10
(array([791]), array([0]))
[[ 7.          7.          7.          7.          0.          0.86100294]]


In [57]:
print xor_opt
print np.amin(histdata), np.amax(histdata)
print len(histdata)
fig = figure()
hist(histdata)
mpld3.display(fig)

0.999999999173
-1.0575251101e-41 0.861002944677
792


In [58]:
z = np.where(array(histdata) < 10**(-15))
print len(z[0])
print len(z[0]) * 10**(-15)
a1 = np.where(array(histdata) > 0.5)
a2 = np.where(array(histdata) > 0.1)
a3 = np.where(array(histdata) > 10**(-9))
a4 = np.where(array(histdata) > 10**(-10))
print a1, a2
print len(a3[0]), len(a4[0])
print histdata[791], histdata[461], histdata[791]+histdata[461]

751
7.51e-13
(array([791]),) (array([461, 791]),)
16 20
0.861002944677 0.127308903764 0.988311848441


In [60]:
histdata = array(histdata)
print a3
print a4
print histdata[[5, 123, 377, 454, 458]]
print sum(histdata[[5, 123, 377, 454, 458]])
print len(np.where(histdata > 10**(-11))[0])
print sum(histdata[[19, 195, 245, 670, 781]])
print 10**(-9) - (1 - xor_opt)

(array([ 20,  54,  55, 124, 125, 249, 250, 251, 459, 460, 461, 783, 788,
       789, 790, 791]),)
(array([  5,  20,  54,  55, 123, 124, 125, 249, 250, 251, 454, 458, 459,
       460, 461, 783, 788, 789, 790, 791]),)
[  4.70900310e-10   4.03271048e-10   0.00000000e+00   8.97966287e-10
   2.93682337e-10]
2.06581998118e-09
23
1.34789725183e-10
1.72646147905e-10


In [128]:
%%time
downbit = upbit = xorbit = 4*10**4/3
N = 4

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 26.9
rSNRdu = 0.7
rSNR3 = 0.8
xor_opt, histdata = xor_analysis_new(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

CPU times: user 613 ms, sys: 49.7 ms, total: 662 ms
Wall time: 741 ms


/Users/leahdickstein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in log10


In [132]:
print 1 - xor_opt
histdata = matrix(histdata)
print shape(histdata)
print np.where(histdata[:,-1] > 10**(-14))
print len(np.where(histdata[:,-2] > 10**(-14))[0])

1.01895014648e-09
(126, 7)
(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [134]:
print histdata[np.where(histdata[:,-2] > 10**(-14))[0],:-2]

[[ 1.  1.  1.  1.  0.]
 [ 2.  2.  2.  1.  0.]
 [ 2.  2.  2.  2.  0.]
 [ 3.  3.  2.  2.  1.]
 [ 3.  3.  3.  1.  0.]
 [ 3.  3.  3.  2.  0.]
 [ 3.  3.  3.  3.  0.]
 [ 4.  4.  3.  2.  1.]
 [ 4.  4.  3.  3.  0.]
 [ 4.  4.  3.  3.  1.]
 [ 4.  4.  4.  1.  0.]
 [ 4.  4.  4.  2.  0.]
 [ 4.  4.  4.  3.  0.]
 [ 4.  4.  4.  4.  0.]]


In [100]:
histdata = matrix(histdata)
fig = figure()
plot(range(35), histdata[:,-1])
mpld3.display(fig)

In [116]:
%%time
downbit = upbit = xorbit = 4*10**4/3
N = 10

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 15
rSNRdu = 2.0
rSNR3 = 2.5
xor_opt, histdata = xor_analysis_new(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

CPU times: user 12.2 s, sys: 223 ms, total: 12.4 s
Wall time: 14.9 s


/Users/leahdickstein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in log10


In [84]:
print 1 - xor_opt
histdata = matrix(histdata)
print np.where(histdata[:,-1] > 0.5)[0]
print histdata[3002]
print np.where(histdata[:,-1] > 10**(-11))[0]
print len(np.where(histdata[:,-1] > 10**(-11))[0])

7.72447439346e-10
[3002]
[[ 10.          10.          10.          10.           0.           0.56987308]]
[  19   20   54   55  123  124  125  248  249  250  251  454  457  458  459
  460  461  781  782  783  787  788  789  790  791 1273 1275 1276 1277 1281
 1282 1283 1284 1285 1286 1987 1988 1989 1990 1991 1996 1997 1998 1999 2000
 2001 2987 2988 2989 2990 2991 2997 2998 2999 3000 3001 3002]
57


In [79]:
fig = figure()
hist(histdata[:,-1])
mpld3.display(fig)

In [114]:
histdata = matrix(histdata)
print np.where(histdata[:,-1] > 10**(-9))[0]
print len(np.where(histdata[:,-1] > 10**(-14))[0])
print '\n'
fig = figure()
plot(range(3003), histdata[:,-1])
mpld3.display(fig)

[  20   54   55  124  125  249  250  251  458  459  460  461  783  788  789
  790  791 1277 1282 1283 1284 1285 1286 1989 1991 1997 1998 1999 2000 2001
 2989 2991 2997 2998 2999 3000 3001 3002]
87




In [125]:
histdata = matrix(histdata)
# print histdata[np.where(histdata[:,-2] > 10**(-11))[0],-1]
fig=figure()
plot(range(3003), log10(histdata[:,-1]), lw=2)
mpld3.display(fig)

In [127]:
histdata = matrix(histdata)
print histdata[np.where(histdata[:,-2] > 10**(-14))[0],:-2]

[[  1.   1.   1.   1.   0.]
 [  2.   2.   2.   1.   0.]
 [  2.   2.   2.   2.   0.]
 [  3.   3.   3.   1.   0.]
 [  3.   3.   3.   2.   0.]
 [  3.   3.   3.   3.   0.]
 [  4.   4.   3.   3.   1.]
 [  4.   4.   4.   1.   0.]
 [  4.   4.   4.   2.   0.]
 [  4.   4.   4.   3.   0.]
 [  4.   4.   4.   4.   0.]
 [  5.   5.   4.   3.   1.]
 [  5.   5.   4.   4.   0.]
 [  5.   5.   4.   4.   1.]
 [  5.   5.   5.   1.   0.]
 [  5.   5.   5.   2.   0.]
 [  5.   5.   5.   3.   0.]
 [  5.   5.   5.   4.   0.]
 [  5.   5.   5.   5.   0.]
 [  6.   6.   5.   3.   1.]
 [  6.   6.   5.   4.   0.]
 [  6.   6.   5.   4.   1.]
 [  6.   6.   5.   5.   0.]
 [  6.   6.   5.   5.   1.]
 [  6.   6.   6.   1.   0.]
 [  6.   6.   6.   2.   0.]
 [  6.   6.   6.   3.   0.]
 [  6.   6.   6.   4.   0.]
 [  6.   6.   6.   5.   0.]
 [  6.   6.   6.   6.   0.]
 [  7.   7.   6.   4.   1.]
 [  7.   7.   6.   5.   0.]
 [  7.   7.   6.   5.   1.]
 [  7.   7.   6.   6.   0.]
 [  7.   7.   6.   6.   1.]
 [  7.   7.   7.   2